# Noa's script for analyzing iEEG data of pyFR, FR1 or catFR

In [153]:
def getBadChannels(pairs,elec_cats,remove_soz_ictal):
    # get the bad channels and soz/ictal/lesion channels from electrode_categories.txt files
    bad_bp_mask = np.zeros(len(pairs))
    if elec_cats != []:
        if remove_soz_ictal == True:
            bad_elecs = elec_cats['bad_channel'] + elec_cats['soz'] + elec_cats['interictal']
        else:
            bad_elecs = elec_cats['bad_channel']
        for row_num in range(0,len(pairs)): 
            labels=pairs.iloc[row_num]['label']
            elec_labels = labels.split('-')
            if elec_labels[0] in bad_elecs or elec_labels[1] in bad_elecs:
                bad_bp_mask[row_num] = 1
    return bad_bp_mask

In [154]:
 def getElecCats(reader):  
    try:
        elec_cats = reader.load('electrode_categories') # contains info about seizure onset zone, interictal spiking, broken leads.
        bad_elec_status=str(len(elec_cats))+' electrode categories'
    except:
        bad_elec_status= 'failed loading electrode categories'
        elec_cats = []
    return elec_cats,bad_elec_status       
    

In [155]:
# Specific subjects that need manual removal of electrodes
def missing_channels(pairs,exp): 
    row_indx=[]
    delete_chans=[]
    missing_chans=[]
    if sub == 'R1350D':
        missing_chans= ['LAHCMD2-LAHCMD3', 'LAHCMD4-LAHCMD5', 'LAHCMD6-LAHCMD7', 'LAHCMD8-LAHCMD1', 'LPHCMD2-LPHCMD3', 'LPHCMD4-LPHCMD5', 'LPHCMD6-LPHCMD7', 'LPHCMD8-LPHCMD1']
    elif sub == 'R1354E':
        missing_chans= ['5Ldm1-5Ldm2', '5Ldm2-5Ldm3', '5Ldm3-5Ldm4', '5Ldm4-5Ldm5', '5Ldm5-5Ldm6', '5Ldm6-5Ldm7', '5Ldm7-5Ldm8', '5Ldm8-5Ldm9', '5Ldm9-5Ldm1', '25Ldm1-25Ldm2', '25Ldm2-25Ldm3', '25Ldm3-25Ldm4', '25Ldm4-25Ldm5', '25Ldm5-25Ldm6', '25Ldm6-25Ldm7', '25Ldm7-25Ldm8', '25Ldm8-25Ldm9', '25Ldm9-25Ldm1']
    elif sub == 'R1368T':
        missing_chans=['LBMicro1-LBMicro2', 'LBMicro2-LBMicro3', 'LBMicro3-LBMicro4', 'LBMicro4-LBMicro5', 'LBMicro5-LBMicro6', 'LBMicro6-LBMicro7', 'LBMicro7-LBMicro8', 'LBMicro8-LBMicro1']
    elif sub=='R1387E':
        missing_chans=['5Ld8-5Ld9', '5Ld10-5Ld11', '5Ld12-5Ld1', '5Ld14-5Ld15', '5Ld16-5Ld13']
    elif sub=='R1393T' and exp=='catFR1':
        missing_chans=['LBMI2-LBMI3', 'LBMI4-LBMI5', 'LBMI6-LBMI7', 'LBMI8-LBMI1', 'LBMI10-LBMI11', 'LBMI12-LBMI13', 'LBMI14-LBMI15', 'LBMI16-LBMI9']
    elif sub == 'R1492J' and exp=='catFR1':
        missing_chans=['LQA7-LQA8']
    elif sub == 'R1525J' and exp=='catFR1':
        missing_chans= ['RQ6-RQ7', 'LC10-LC11']
    for row in range(0,len(pairs)):
        labels=pairs['label'][row]
        if labels in missing_chans:
            delete_chans.append(row)
            row_indx.append(row)
    pairs=pairs.drop(axis=0,labels=delete_chans)
    return pairs,row_indx

In [156]:
# list of subjects that their eeg is misaligned
def correctEEGsubjects(sub):
    import re
    sub_num = [int(s) for s in re.findall(r'\d+',sub)]
    if sub_num[0]>=1397:
        in_list = True
    elif sub in ['R1379E','R1385E','R1387E','R1394E','R1402E','R1404E','R1396T','R1396T','R1395M','R1389J']:
        in_list = True
    else:
        in_list = False
    return in_list

In [178]:
## iEEG data - trial level
# from scipy.io import loadmat

for i in range(0,len(subject_info)):  
    
    if i==240 and exp=='catFR1': #problematic eeg loading..
        removed_subject=removed_subject.append({'subject':sub, 'sessions':sess,'montage':mont,'reason for exclusion':'No hippocampal electrodes'},ignore_index=True)
        continue
    sub = subject_info.iloc[i]['subject']
    exp = subject_info.iloc[i]['experiment']
    sess= int(subject_info.iloc[i]['session']) 
    mont= int(subject_info.iloc[i]['montage'])
    loc=  int(subject_info.iloc[i]['localization']) # mostly 0  
    assert len(set(subject_info.iloc[i]['trials for analysis']) & set(subject_info.iloc[i]['<>verbalization_trials'] ))==0 # make sure there's no overlapping trials
    trials= subject_info.iloc[i]['trials for analysis'] +subject_info.iloc[i]['<>verbalization_trials']   
    
    reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)
    evs = reader.load('task_events')
    evs=add_output_position(evs)
      
    if len(trials)==0:
        continue
    chosen_data = evs.loc[trials]
    in_list=correctEEGsubjects(sub)
    if in_list==True:
        chosen_data=correctEEGoffset(sub,sess,exp,reader,chosen_data) # correcting eeg-behavior misalignment
    
    # load bad electrodes information  
    remove_soz_ictal=False #True # decide if True (remove seizure onset zone and interictal) or False (exclude only bad channels)
    elec_cats,bad_elec_status=getElecCats(reader)
    try:
        try:
            if exp=='pyFR':
                tal_path='/data/eeg/'+sub+'/tal/'+sub+'_talLocs_database_bipol.mat'
                df = pd.DataFrame(loadmat(tal_path)['bpTalStruct'][0])
                channel_info=[[df['channel'][i][0][0],df['channel'][i][0][1],df['tagName'][i][0],df['Loc5'][i][0]] for i in range(0,len(df))]
                pairs=pd.DataFrame(channel_info, columns=['contact_1','contact_2','label','stein.region'])

            elif exp=='FR1' or exp=='catFR1' :   
                pairs = reader.load('pairs')
        except:
            from ptsa.data.readers import TalReader
            tal_path='/data/eeg/'+sub+'/tal/'+sub+'_talLocs_database_bipol.mat'
            tr=TalReader(tal_path)
            bp=tr.get_bipolar_pairs().astype([('ch0','U8'),('ch1','U8')]) #list of bipolar pairs
            pd.set_option('max_columns',None)
            loc1=tr.read()
            df=pd.DataFrame.from_records(loc1.tolist(),columns=np.array(loc1.dtype.names))
            #loc_df.head()
            channel_info=[[df['channel'][i][0],df['channel'][i][1],df['tagName'][i],df['Loc5'][i]] for i in range(0,len(df))]
            pairs=pd.DataFrame(channel_info, columns=['contact_1','contact_2','label','stein.region'])

        pairs,rows=missing_channels(pairs,exp) # handles missing channels in the data
        
        #Grab some EEG 
        buf = 1500  #to remove edge effects during wavelet convolution
        eeg = reader.load_eeg(events=chosen_data, rel_start=-4500-buf, rel_stop=1000+buf, scheme=pairs)
    except:
        print('***sub:',sub,'exp:',exp,' eeg data is not loading')
        removed_subject=removed_subject.append({'subject':sub, 'sessions':sess,'montage':mont,'reason for exclusion':'eeg loading problem'},ignore_index=True)
        continue
    
    if sub not in removed_subject[removed_subject['reason for exclusion']== 'eeg loading problem']['subject'].unique():  
        # find electrodes in the requested ROI
        try:
            localizations=reader.load('localizations')
        except:
            localizations=[]
        from regionalizationModule import *
        elect_regions=get_elec_regions(localizations,pairs) # John's script        

        
        mat=np.array(zeros(len(elect_regions[0])))
        for word in region_list:
            mat=mat+((elect_regions[0]==word)*1)
        hipp_index=np.array(mat)>0 # bool mask

        
        # find bad electrodes
        bad_bp_mask = getBadChannels(pairs,elec_cats,remove_soz_ictal)
        #concat hippocampus electrodes with good electrodes:
        good_bp_mask_bool=bad_bp_mask==0 # good electrodes boolean
        good_bp_hipp_mask = good_bp_mask_bool & hipp_index
        assert sum(good_bp_hipp_mask) <= sum(hipp_index)

        hipp_electrodes=hipp_electrodes.append({'subject':sub,'session':sess,'montage':mont,'localization':loc,'number of hipp electrodes': sum(hipp_index),
                                               'number of good hipp electrodes':sum(good_bp_hipp_mask)},ignore_index=True)

        if sum(good_bp_hipp_mask)>0:  # if at least one electrode exists  
            
            # sample rate should be at least 250Hz
            sr = round(eeg.samplerate)
            if sr <250:
                print('***sub:',sub,'exp:',exp,' sample rate is lower than 250 Hz: sr is: ',sr)
                removed_subject=removed_subject.append({'subject':sub, 'sessions':sess,'montage':mont,'reason for exclusion':'sample rate is low','sample rate':sr},ignore_index=True)
            elif sr >=250:
                if sr %100 !=0:
                    sr=sr-sr%100
                    eeg=eeg.resample(sr)
                eeg_ptsa = eeg.to_ptsa()

                from ptsa.data.filters import ButterworthFilter
                from ptsa.data.filters import MorletWaveletFilter

                if sub.startswith('FR'): # Freiburg subjects' line noise is 50Hz
                    freq_range = [48., 52.]
                else:
                    freq_range = [58., 62.]

                b_filter = ButterworthFilter(timeseries=eeg_ptsa, freq_range=freq_range, filt_type='stop', order=4)
                eeg_filtered = b_filter.filter()

                # Zero values in the eeg data will become -inf under log transform. 
                bad_channels=[]
                good_channels=[]
                for channel in range(0,eeg_filtered.shape[1]):
                    if all(np.array(eeg_filtered[:,channel,:])==0): # bad electrodes with zeros
                        bad_channels.append(channel)
                for chan in range(0,eeg_filtered.shape[1]):
                    if chan not in bad_channels:
                        good_channels.append(chan)
                eeg_filtered = eeg_filtered[:,good_channels,:]

                #Get spectral power
                myfreqs = np.logspace(np.log10(2), np.log10(100), 46)  #2-100 Hz, 46 frequencies
                wf = MorletWaveletFilter(timeseries=eeg_filtered, freqs=myfreqs, width=6, output='power') 
                pow_wavelet = wf.filter()   #output is freqs, events, elecs, and time
                pows = pow_wavelet[:, :, :, int((buf/1000.)*sr):-1*int((buf/1000.)*sr)]  #remove buffer period
                pows = np.log10(pows) # get log of powers. # powes time is 2750 samples, corresponding to 2750/500=5,500ms. # in some versions use:np.log10(pows.data) instead.

                    # split to events:
                correct_recall_serial_index=[]
                ELIs_serial_index=[]
                PLIs_serial_index=[]
                All_intrusions_serial_index=[]
                deliberation_serial_index=[]
                related_intrusions_serial_index=[]
                nonrelated_intrusions_serial_index=[]
                if any(chosen_data.columns=='item_name'): # in the pyFR and FR/catFR this column has a different name 
                        item='item_name'
                elif any(chosen_data.columns=='item'):
                        item='item'
                for ii in range(0,len(chosen_data)):
                    intrusion_value=chosen_data['intrusion'].iloc[ii]
                    semantic_boolean=chosen_data['semantic_relatedness'].iloc[ii]
                    word=chosen_data[item].iloc[ii]
                    if intrusion_value==0:
                        correct_recall_serial_index.append(ii)
                        deliberation_serial_index.append(ii)
                    elif intrusion_value==-1 and word!='<>': 
                        ELIs_serial_index.append(ii)
                        All_intrusions_serial_index.append(ii)
                        deliberation_serial_index.append(ii)
                        if semantic_boolean==1:
                            related_intrusions_serial_index.append(ii)
                        elif semantic_boolean==0:
                            nonrelated_intrusions_serial_index.append(ii)
                            
                    elif intrusion_value==-1 and word=='<>': 
                        deliberation_serial_index.append(ii)
                    elif intrusion_value>0:
                        PLIs_serial_index.append(ii)
                        All_intrusions_serial_index.append(ii)
                        deliberation_serial_index.append(ii)
                        if semantic_boolean==1:
                            related_intrusions_serial_index.append(ii)
                        elif semantic_boolean==0:
                            nonrelated_intrusions_serial_index.append(ii)
  
                assert len(chosen_data)>=len(PLIs_serial_index+ELIs_serial_index+correct_recall_serial_index),'there is an error in the calculation of events serial index' 
                assert len(All_intrusions_serial_index)==len(ELIs_serial_index)+len(PLIs_serial_index),'error in All_intrusions_serial_index calculation'
                assert len(chosen_data)==len(deliberation_serial_index),'error in deliberation_serial_index calculation'

                excluded_channels=excluded_channels.append({'subject':sub,'session':sess,'montage':mont,'localization':loc,'number of hipp electrodes': sum(hipp_index),
                                                            'number of good ROI electrodes':sum(good_bp_hipp_mask),
                                                            'channels without data': bad_channels,
                                                            'number of correct recalls': len(correct_recall_serial_index),
                                                           'number of intrusions': len(All_intrusions_serial_index),
                                                           'number of deliberations': len(deliberation_serial_index),
                                                           'number of ELI':len(ELIs_serial_index),
                                                           'number of PLI':len(PLIs_serial_index),
                                                           'loading bad electrodes': bad_elec_status,
                                                           'sample rate': sr},ignore_index=True)

                #Reshape array into 100 ms bins with a 50ms sliding window
                time_window=int(100/1000*sr) # convert 100ms to samples
                sliding_time=int(50/1000*sr) # convert 50ms to samples
                time_windows=[(k,k+time_window) for k in range(0,pows.shape[3]+1-time_window+1,sliding_time)] # 50ms sliding window is 25 samples
                if len(time_windows)==108: # i=49 terminated before 2750 sample
                    time_windows.append((time_windows[-1][0]+sliding_time,pows.shape[3]))
                avg_pows=np.zeros([pows.shape[0],pows.shape[1],pows.shape[2],len(time_windows)])*np.nan # initialize to nans
                index=0
                for start,end in time_windows:
                    avg_pows[:,:,:,index]=np.mean(pows[:,:,:,start:end],axis=3)
                    index=index+1        

                average_events_time=np.mean(avg_pows,axis=(1,3))
                standard_deviation_event_time=np.std(avg_pows,axis=(1,3)) # here I compute std across events and time
                z_pows=np.zeros([avg_pows.shape[0],avg_pows.shape[1],avg_pows.shape[2],avg_pows.shape[3]])*np.nan
                for freq in range(0,average_events_time.shape[0]):
                    for electrode in range(0,average_events_time.shape[1]):
                        z_pows[freq,:,electrode,:]=(avg_pows[freq,:,electrode,:]-average_events_time[freq][electrode])/standard_deviation_event_time[freq][electrode]

                if any(isnan(z_pows)):
                    print('NaNs in subject: ',sub, 'session: ', sess, 'montage: ',mont, ' index loop: ',i)


        #         #SPLIT THE DATA TO EVENTS and average them
                pows_correct_recall=z_pows[:,correct_recall_serial_index,:,:] # avergae over all events
                pows_PLI=z_pows[:,PLIs_serial_index,:,:]
                pows_ELI=z_pows[:,ELIs_serial_index,:,:]
                pows_all_intrusions= z_pows[:,All_intrusions_serial_index,:,:]
                pows_deliberation = z_pows[:,deliberation_serial_index,:,:] 
                pows_related_intrusions = z_pows[:,related_intrusions_serial_index,:,:] 
                pows_nonrelated_intrusions = z_pows[:,nonrelated_intrusions_serial_index,:,:] 

                # Average over ROI electrodes
                if len(bad_channels)>0:
                    hipp_index=numpy.delete(hipp_index, bad_channels)
                    good_bp_hipp_mask=numpy.delete(good_bp_hipp_mask, bad_channels)
                if sum(good_bp_hipp_mask)>0:
                    correct_recall_meanPower_hippo=np.mean(pows_correct_recall[:,:,good_bp_hipp_mask,:],axis=2)
                    PLI_meanPower_hippo=np.mean(pows_PLI[:,:,good_bp_hipp_mask,:],axis=2)
                    ELI_meanPower_hippo=np.mean(pows_ELI[:,:,good_bp_hipp_mask,:],axis=2)
                    all_intrusions_meanPower_hippo=np.mean(pows_all_intrusions[:,:,good_bp_hipp_mask,:],axis=2)
                    deliberation_meanPower_hippo=np.mean(pows_deliberation[:,:,good_bp_hipp_mask,:],axis=2)
                    related_intrusions_meanPower_roi=np.mean(pows_related_intrusions[:,:,good_bp_hipp_mask,:],axis=2)
                    nonrelated_intrusions_meanPower_roi=np.mean(pows_nonrelated_intrusions[:,:,good_bp_hipp_mask,:],axis=2)


                    try:
                        assert deliberation_meanPower_hippo.shape[2]== 109,'deliberation dimensions inaccurate for subject '+sub+' index: '+str(i)+' session: '+str(sess)
                        assert all_intrusions_meanPower_hippo.shape[2] == 109,'all_intrusions dimensions inaccurate for subject '+sub+' index: '+str(i)+' session: '+str(sess)
                        assert correct_recall_meanPower_hippo.shape[2] == 109,'correct_recall dimensions inaccurate for subject '+sub+' index: '+str(i)+' session: '+str(sess)
                    except:
                        print(sub, 'session:', sess, 'montage:', mont, 'time dimension is not 109. Check it out, index:', i)

                    # the output has: 46 frequencies x trials x 109 time windows    
                    correct_recall_hipp_pows.append(correct_recall_meanPower_hippo)
                    PLI_num=sum(subject_info[subject_info['subject']==sub]['number of prior-list intrusions'])
                    ELI_num=sum(subject_info[subject_info['subject']==sub]['number of extra-list intrusions'])
                    if PLI_num>=5 and ELI_num>=5:
                        PLI_hipp_pows.append(PLI_meanPower_hippo)
                        ELI_hipp_pows.append(ELI_meanPower_hippo)
                    all_intrusions_hipp_pows.append(all_intrusions_meanPower_hippo)
                    deliberation_hipp_pows.append(deliberation_meanPower_hippo)
                    related_intrusions_roi_pows.append(related_intrusions_meanPower_roi)
                    nonrelated_intrusions_roi_pows.append(nonrelated_intrusions_meanPower_roi)
                    
                    # record output position of chosen events AND semantic similarity:
                    evs_counter=evs_counter+1
                    correct_recall_output_position.append(chosen_data.iloc[correct_recall_serial_index]['output position'])
                    all_intrusions_output_position.append(chosen_data.iloc[All_intrusions_serial_index]['output position'])
                    deliberation_output_position.append(chosen_data.iloc[deliberation_serial_index]['output position'])
                    related_intrusions_position.append(chosen_data.iloc[related_intrusions_serial_index]['output position'])
                    nonrelated_intrusions_position.append(chosen_data.iloc[nonrelated_intrusions_serial_index]['output position'])
                    
                    if exp == 'catFR1':
                        correct_recall_semantic_similarity.append(chosen_data.iloc[correct_recall_serial_index]['semantic_similarity'])
                        all_intrusions_semantic_similarity.append(chosen_data.iloc[All_intrusions_serial_index]['semantic_similarity'])
                        deliberation_semantic_similarity.append(chosen_data.iloc[deliberation_serial_index]['semantic_similarity'])
                        related_intrusions_semantic_similarity.append(chosen_data.iloc[related_intrusions_serial_index]['semantic_similarity'])
                        nonrelated_intrusions_semantic_similarity.append(chosen_data.iloc[nonrelated_intrusions_serial_index]['semantic_similarity'])

                        correct_recall_semantic_relatedness.append(chosen_data.iloc[correct_recall_serial_index]['semantic_relatedness'])
                        all_intrusions_semantic_relatedness.append(chosen_data.iloc[All_intrusions_serial_index]['semantic_relatedness'])
                        deliberation_semantic_relatedness.append(chosen_data.iloc[deliberation_serial_index]['semantic_relatedness'])
                        related_intrusions_semantic_relatedness.append(chosen_data.iloc[related_intrusions_serial_index]['semantic_relatedness'])
                        nonrelated_intrusions_semantic_relatedness.append(chosen_data.iloc[nonrelated_intrusions_serial_index]['semantic_relatedness'])

                    if PLI_num>=5 and ELI_num>=5:
                        intusion_counter=intusion_counter+1
                        PLI_output_position.append(chosen_data.iloc[PLIs_serial_index]['output position'])
                        ELI_output_position.append(chosen_data.iloc[ELIs_serial_index]['output position'])
                        PLI_ELIsubs_inserted_to_analysis=PLI_ELIsubs_inserted_to_analysis.append({'subject':sub,'session':sess,'montage':mont,'localization':loc},ignore_index=True)
                        if exp == 'catFR1':
                            PLI_semantic_similarity.append(chosen_data.iloc[PLIs_serial_index]['semantic_similarity'])
                            ELI_semantic_similarity.append(chosen_data.iloc[ELIs_serial_index]['semantic_similarity'])
                            PLI_semantic_relatedness.append(chosen_data.iloc[PLIs_serial_index]['semantic_relatedness'])
                            ELI_semantic_relatedness.append(chosen_data.iloc[ELIs_serial_index]['semantic_relatedness'])
                            
                    assert len(correct_recall_hipp_pows[evs_counter][0])==len(correct_recall_output_position[evs_counter]) # will work only if loop begins with 0 (i above)
                    if PLI_num>=5 and ELI_num>=5:
                        assert len(PLI_hipp_pows[intusion_counter][0])==len(PLI_output_position[intusion_counter])
                        assert len(ELI_hipp_pows[intusion_counter][0])==len(ELI_output_position[intusion_counter])
                    assert len(all_intrusions_hipp_pows[evs_counter][0])==len(all_intrusions_output_position[evs_counter])
                    assert len(deliberation_hipp_pows[evs_counter][0])==len(deliberation_output_position[evs_counter])
                    
                    subs_inserted_to_analysis=subs_inserted_to_analysis.append({'subject':sub,'session':sess,'montage':mont,'localization':loc},ignore_index=True)
        else:
            print('subject',sub, 'session', sess, 'has no electrodes in the hippocampus and was not included in the analysis')
            removed_subject=removed_subject.append({'subject':sub, 'sessions':sess,'montage':mont,'reason for exclusion':'No hippocampal electrodes'},ignore_index=True)
    print('Progress:', int((i+1)/len(subject_info)*100),'% done, subject: ',sub)

/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /data10/RAM/subjects/R1016M/docs/electrode_categories.txt
/scratch/pwanda/electrode_categories/electrode_categories_R1016M.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1016M/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1016M/tal/R1016M_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/noaherz/cmlreaders/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could

subject R1016M session 0 has no electrodes in the hippocampus and was not included in the analysis
Progress: 0 % done, subject:  R1016M


/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /data10/RAM/subjects/R1016M/docs/electrode_categories.txt
/scratch/pwanda/electrode_categories/electrode_categories_R1016M.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files fou

subject R1016M session 1 has no electrodes in the hippocampus and was not included in the analysis
Progress: 0 % done, subject:  R1016M


/home1/noaherz/.conda/envs/cml3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /data10/RAM/subjects/R1021D/docs/electrode_categories.txt
/scratch/pwanda/electrode_categories/electrode_categories_R1021D.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1021D/localizations/0/montages/0/neuroradiology/current_processed/pairs.json
/data/eeg/R1021D/tal/R1021D_talLocs_database_bipol.mat returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)
/home1/noaherz/cmlreaders/cmlreaders/readers/electrodes.py:236: MissingCoordinatesWarning: Could

...

subject R1401J session 0 has no electrodes in the hippocampus and was not included in the analysis
Progress: 53 % done, subject:  R1401J


/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1401J/experiments/catFR1/sessions/1/ephys/current_processed/sources.json
/data/eeg/R1401J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


subject R1401J session 1 has no electrodes in the hippocampus and was not included in the analysis
Progress: 53 % done, subject:  R1401J


/home1/noaherz/cmlreaders/cmlreaders/path_finder.py:239: MultiplePathsFoundWarning: Multiple files found: /protocols/r1/subjects/R1401J/experiments/catFR1/sessions/2/ephys/current_processed/sources.json
/data/eeg/R1401J/eeg.noreref/params.txt returning the first file found
  warnings.warn(msg, MultiplePathsFoundWarning)


subject R1401J session 2 has no electrodes in the hippocampus and was not included in the analysis
Progress: 54 % done, subject:  R1401J
subject R1403N session 0 has no electrodes in the hippocampus and was not included in the analysis
Progress: 54 % done, subject:  R1403N
subject R1403N session 1 has no electrodes in the hippocampus and was not included in the analysis
Progress: 54 % done, subject:  R1403N
subject R1403N session 2 has no electrodes in the hippocampus and was not included in the analysis
Progress: 54 % done, subject:  R1403N
subject R1403N session 4 has no electrodes in the hippocampus and was not included in the analysis
Progress: 55 % done, subject:  R1403N
subject R1404E session 0 has no electrodes in the hippocampus and was not included in the analysis
Progress: 55 % done, subject:  R1404E
subject R1404E session 1 has no electrodes in the hippocampus and was not included in the analysis
Progress: 55 % done, subject:  R1404E
subject R1405E session 0 has no electrode